<a href="https://colab.research.google.com/github/IceMerman/NN_validation/blob/master/notebooks/exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Database
[Available at Gdrive](https://drive.google.com/drive/folders/1siq0kTyjfqAUDNefl4sXw8epz-fk-_gb?usp=sharing)

# Creation


The database was created using [MatPower](https://matpower.org/), the source can be found at [GitHub](https://github.com/IceMerman/NN_validation/tree/master/Demanda)


In [ ]:
# libraries
from scipy.io import loadmat
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# load database
mat = loadmat('/content/db.mat')
labels = mat['issecure']
VM = mat['VM']
VA = mat['VA']
# Reshape sload half of the data is active power, the other one is reactive power
SLOAD = mat['SLOAD']


EPOCHS = 2

In [ ]:
samples, powers = SLOAD.shape
PLOAD = SLOAD[:, :powers//2]
QLOAD = SLOAD[:, powers//2:]

In [ ]:
def sign(fval: float)-> str:
  if fval >= 0:
    return "+"
  else:
    return "-"

def print_rect(id_experiment: int = 0) -> None:
  for i,power in enumerate(PLOAD[id_experiment,:]):
    reactive = QLOAD[id_experiment,i]
    s_reactive = sign(reactive)
    print(f'{power:.2f} {s_reactive} j {abs(reactive):.2f}')

In [ ]:
# Neural net
class Net(nn.Module):
  def __init__(self, first_layers_size):
    super().__init__()
    self.fc1 = nn.Linear(first_layers_size, first_layers_size//2)
    self.fc2 = nn.Linear(first_layers_size//2, 2*first_layers_size)
    self.fc3 = nn.Linear(2*first_layers_size, (first_layers_size*3)//2)
    self.fc4 = nn.Linear((first_layers_size*3)//2, first_layers_size//2)
    self.fc5 = nn.Linear(first_layers_size//2, 1)
    
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = self.fc5(x)
    return x

net = Net(powers//2)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
trainloader = []

for epoch in range(EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')